In [12]:
# using Pkg
# Pkg.add("LinearAlgebra")
# Pkg.add("GLPK")
# Pkg.add("Convex")
# Pkg.add("JuMP")
# Pkg.add("GLPKMathProgInterface")
# Pkg.add("Cbc")
# Pkg.add("Clp")
# Pkg.add("CPLEX")
# Pkg.add("Gurobi")

In [13]:
using LinearAlgebra, GLPK, Convex

Given the chemical: $$C_{11}H_{26}O_6N_2P_2S_2Cl_2$$
Accurate masses of atoms:
<br>$C:12.000000$ $\quad$ $H:1.0078250$ $\quad$ $O:15.994915$ $\quad$ $N:14.003074$ $\quad$ $P:30.973762$ $\quad$ $S:31.972071$ $\quad$ $Cl:34.968853$
<br>
<br>Introduce variables:
$$x = \pmatrix{x_C \cr x_H \cr x_O \cr x_N \cr x_P \cr x_S \cr x_{Cl}} \quad A = \pmatrix{12.000000 \cr 1.0078250 \cr 15.994915 \cr 14.003074 \cr 30.973762 \cr 31.972071 \cr 34.968853} \quad f = \pmatrix{11 \cr 26 \cr 6 \cr 2 \cr 2 \cr 2 \cr 2}$$
<br>Given a fragment with mass $c$, we want to determin a linear combination of atoms such that $m/z$ is most close to value $c$
<br>
<br>The question is equivalent to solving optimization problem:
$$\min_{x \in \mathbb{N}^7} |a^Tx-c|$$
Instead, we solve:
$$\begin{array}{ll}
\min_{x \in \mathbb{R}^7} & |a^Tx-c|
\\\text{s.t.} & x\geq 0
\\ & x\leq f
\end{array}$$
Which is equivalent to the problem with auxillary variable:
$$\begin{array}{ll}
\min_{t \in \mathbb{R}} & t
\\\text{s.t.} & t\geq ax-c
\\ & t\geq -(ax-c)
\\ & x\geq 0
\\ & x\leq f
\end{array}$$
With one more step of transformation:
$$\begin{array}{ll}
\min_{t \in \mathbb{R}} & t
\\\text{s.t.} & t-ax\geq -c
\\ & t+ax\geq c
\\ & x\geq 0
\\ & -x\geq -f
\end{array}$$
With matix expression, we have:
$$\hat{A} = \pmatrix{-A^T & 1 \cr A^T & 1 \cr I_{7\times 7} & 0_{7\times 1} \cr -I_{7\times 7} & 0_{7\times 1}} \quad \hat{x} = \pmatrix{x \cr t} \quad b = \pmatrix{-c \cr c \cr 0_{7\times 1} \cr -f}$$
The optimization problem is will be equivalent to:
$$\begin{array}{ll}
\min & \pmatrix{0 & 1}\hat{x}
\\\text{s.t.} & \hat{A}\hat{x}-b\geq 0 
\end{array}$$

Now generate variables $A, f$, test for $C_8H_{11} O_4 N_2 P_1 S_2 Cl_1$, $c=328.9592$

In [14]:
A = [12 1.007825 15.994915 14.003074 30.973762 31.972071 34.968853]'
f = [11 26 6 2 2 2 2]
c = 328.9592;

In [15]:
x = Variable(7+1)
Im = diagm(0=>fill(1, 7))
A_h = [-A' 1;
        A' 1;
        Im zeros(7,1);
       -Im zeros(7,1)]
b = [-c;
      c;
      zeros(7,1);
      -f'];

In [16]:
problem = minimize([zeros(1,7) 1]*x, [A_h * x >= b])
solve!(problem, GLPK.Optimizer)
x.value

8×1 Array{Float64,2}:
 0.7628491666666687
 0.0
 6.0
 2.0
 2.0
 2.0
 2.0
 0.0

Therefore we cannot simply try to solve the problem by soving $x\in\mathbb{R}^7$, now we need to solve the mixed integer programming problem

For the previous problem, we need to add a new constrain that $x\in\mathbb{Z}^7$
$$\begin{array}{ll}
\min_{t \in \mathbb{R}} & t
\\\text{s.t.} & t-ax\geq -c
\\ & t+ax\geq c
\\ & x\geq 0
\\ & -x\geq -f
\\ & x \in\mathbb{Z}
\end{array}$$
With standarization to standard form:
$$\begin{array}{ll}
\min_{t \in \mathbb{R}} & t
\\\text{s.t.} & t-ax-s_1 = -c
\\ & t+ax-s_2 = c
\\ & x+s = f
\\ & x, s\geq 0
\\ & s_1, s_2, t \geq 0
\\ & x, s \in\mathbb{Z}
\\ & s_1, s_2, t \in\mathbb{R}
\end{array}$$

Introduce variables $y,z$, with matrix expression:
$$\hat{A}=\pmatrix{-A^T & 0_{1\times 7} & -1 & 0 & 1 \cr A^T & 0_{1\times 7} & 0 & -1 & 1\cr
             I_7 & I_7 & 0_{7\times 1} & 0_{7\times 1} & 0_{7\times 1}}
\quad\hat{x}=\pmatrix{y\cr z}=\pmatrix{x\cr s\cr s_1\cr s_2\cr t}
\quad b=\pmatrix{-c\cr c\cr f}$$
Introducint two variables $d_1=0_{14\times1}, d_2=\pmatrix{0&0&1}$, the question is equivalent to standard form:
$$\begin{array}{ll}
\min & d_1y+d_2z
\\\text{s.t.} & \hat{A}\hat{x}=b
\\ & y\geq0, z\geq0
\\ & y\in\mathbb{Z}, z\in\mathbb{R}
\end{array}$$

In [17]:
A_h = [-A' zeros(1,7) -1 0 1;
        A' zeros(1,7) 0 -1 0;
        Im Im zeros(7,1) zeros(7,1) zeros(7,1)]
b = [-c; c; f']
d = [zeros(1,16) 1];

In [18]:
using JuMP
using GLPKMathProgInterface

In [19]:
model = Model()
set_optimizer(model, GLPK.Optimizer)
y = @variable(model, [1:14], base_name="y", Int);
z = @variable(model, [1:3], base_name="z");
@objective(model, Min, z[3]);

In [20]:
for i in 1:14
    @constraint(model, y[i]>=0)
end
for i in 1:3
    @constraint(model, z[i]>=0)
end

In [21]:
@constraint(model, A_h[:, 1:14]*y + A_h[:, 15:17]*z.==b);

In [22]:
model;

In [23]:
optimize!(model);

In [24]:
optimal_solution = value.(y)

14-element Array{Float64,1}:
  9.0
 13.0
  1.0
  2.0
  2.0
  1.0
  2.0
  2.0
 13.0
  5.0
  0.0
  0.0
  1.0
  0.0

As we can see, the result will be $$C_9 H_{13} O_1 N_2 P_2 S_1 Cl_2$$there has already matched with $m/z=329$, and now we need to add additional constrains that restrict the ratio of each atom.

The additional mass restrictions is:
$$0.2\leq H/C\leq 3.1\\
  0\leq O/C\leq 1.2\\
  0\leq N/C\leq 1.3\\
  0\leq P/C\leq 0.3\\
  0\leq S/C\leq 0.8\\
  0\leq Cl/C\leq 0.8\\
  0\leq hetero/C\leq 2.27$$

Now our problem is:
$$\begin{array}{ll}
\min_{t \in \mathbb{R}} & t
\\\text{s.t.} & t-ax-s_1 = -c
\\ & t+ax-s_2 = c
\\ & x+s = f
\\ & x, s\geq 0
\\ & s_1, s_2, t \geq 0
\\ & x, s \in\mathbb{Z}
\\ & s_1, s_2, t \in\mathbb{R}
\end{array}$$
New restrictions will be expressed as:
$$\begin{array}{ll}
  \frac{x_2}{x_1}\geq0.2 & x_2\geq0.2x_1 & x_2-0.2x_1\geq0\\
  \frac{x_2}{x_1}\leq3.1 & x_2\leq3.1x_1 & x_2-3.1x_1\leq0\\
  \frac{x_3}{x_1}\leq1.2 & x_3\leq1.2x_1 & x_3-1.2x_1\leq0\\
  \frac{x_4}{x_1}\leq1.3 & x_4\leq1.3x_1 & x_4-1.3x_1\leq0\\
  \frac{x_5}{x_1}\leq0.3 & x_5\leq0.3x_1 & x_5-0.3x_1\leq0\\
  \frac{x_6}{x_1}\leq0.8 & x_6\leq0.8x_1 & x_6-0.8x_1\leq0\\
  \frac{x_7}{x_1}\leq0.8 & x_7\leq0.8x_1 & x_7-0.8x_1\leq0\\
  \frac{hetero}{x_1}\leq2.27 & hetero\leq2.27x_1 & hetero-2.27x_1\leq0
\end{array}
$$
With surplus and slack, we can transform the restirstions to:
$$\begin{array}{ll}
    x_2-0.2x_1-s_3=0\\
    x_2-3.1x_1+s_4=0\\
    x_3-1.2x_1+s_5=0\\
    x_4-1.3x_1+s_6=0\\
    x_5-0.3x_1+s_7=0\\
    x_6-0.8x_1+s_8=0\\
    x_7-0.8x_1+s_9=0\\
    x_3+x_4+x_5+x_6+x_7-2.27x_1+s_{10}=0\\
    s_3\geq0\\
    s_4\geq0\\
    s_5\geq0\\
    s_6\geq0\\
    s_7\geq0\\
    s_8\geq0\\
    s_9\geq0\\
    s_{10}\geq0
\end{array}$$

Now the problem is equivalent to:
$$\begin{array}{ll}
\min_{t \in \mathbb{R}} & t
\\\text{s.t.} & t-ax-s_1 = -c
\\ & t+ax-s_2 = c
\\ & x+s = f
\\ & x_2-0.2x_1-s_3=0
\\ & x_2-3.1x_1+s_4=0
\\ & x_3-1.2x_1+s_5=0
\\ & x_4-1.3x_1+s_6=0
\\ & x_5-0.3x_1+s_7=0
\\ & x_6-0.8x_1+s_8=0
\\ & x_7-0.8x_1+s_9=0
\\ & x_3+x_4+x_5+x_6+x_7-2.27x_1+s_{10}=0
\\ & x, s\geq 0
\\ & s_1,..., s_{10}, t \geq 0
\\ & x, s \in\mathbb{Z}
\\ & s_1,..., s_{10}, t \in\mathbb{R}
\end{array}$$

With matrix expression:
$$\hat{A}=\pmatrix{-A^T & 0_{1\times 7} & -1 & 0 &\cdots & 0 & 1 \cr A^T & 0_{1\times 7} & 0 & -1 &\cdots & 0 & 1\cr
             I_7 & I_7 & 0_{7\times 1} & 0_{7\times 1}&\cdots &\cdots & 0_{7\times 1}}
\quad\hat{x}=\pmatrix{y\cr z}=\pmatrix{x\cr s\cr s_1\cr \vdots \cr s_{10}\cr t}
\quad b=\pmatrix{-c\cr c\cr f\cr 0_{8\times1}}\\
\tilde{A}=\pmatrix{-0.2 & 1 & 0 & 0 & 0 & 0 & 0 & 0_{1\times7} & 0 & 0 & -1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \cr
                   -3.1 & 1 & 0 & 0 & 0 & 0 & 0 & 0_{1\times7} & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \cr
                   -1.2 & 0 & 1 & 0 & 0 & 0 & 0 & 0_{1\times7} & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\cr
                   -1.3 & 0 & 0 & 1 & 0 & 0 & 0 & 0_{1\times7} & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\cr
                   -0.3 & 0 & 0 & 0 & 1 & 0 & 0 & 0_{1\times7} & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\cr
                   -0.8 & 0 & 0 & 0 & 0 & 1 & 0 & 0_{1\times7} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\cr
                   -0.8 & 0 & 0 & 0 & 0 & 0 & 1 & 0_{1\times7} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\cr
                   -2.27 & 0 & 1 & 1 & 1 & 1 & 1 & 0_{1\times7} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\cr}$$
The question is equivalent to the standard form:
$$\begin{array}{ll}
\min & d_1y+d_2z
\\\text{s.t.} & \pmatrix{\hat{A}\cr\tilde{A}}\hat{x}=b
\\ & y\geq0, z\geq0
\\ & y\in\mathbb{Z}, z\in\mathbb{R}
\end{array}$$

In [25]:
using LinearAlgebra
using JuMP
using GLPKMathProgInterface

In [26]:
f = [11 26 6 2 2 2 2] # parameter
c = 328.9592 # parameter

a = [12 1.007825 15.994915 14.003074 30.973762 31.972071 34.968853]'
I = diagm(0=>fill(1, 7))
z17 = zeros(1,7)
z71 = zeros(7,1)
z81 = zeros(8,1)
    
A   = [-a'                            z17  -1  0   0   0   0   0   0   0   0   0   1;
        a'                            z17  0   -1  0   0   0   0   0   0   0   0   1;
        I                             I    z71 z71 z71 z71 z71 z71 z71 z71 z71 z71 z71;
        -0.2  1   0   0   0   0   0   z17  0   0   -1  0   0   0   0   0   0   0   0;
        -3.1  1   0   0   0   0   0   z17  0   0   0   1   0   0   0   0   0   0   0;
        -1.2  0   1   0   0   0   0   z17  0   0   0   0   1   0   0   0   0   0   0;
        -1.3  0   0   1   0   0   0   z17  0   0   0   0   0   1   0   0   0   0   0;
        -0.3  0   0   0   1   0   0   z17  0   0   0   0   0   0   1   0   0   0   0;
        -0.8  0   0   0   0   1   0   z17  0   0   0   0   0   0   0   1   0   0   0;
        -0.8  0   0   0   0   0   1   z17  0   0   0   0   0   0   0   0   1   0   0;
        -2.27 0   1   1   1   1   1   z17  0   0   0   0   0   0   0   0   0   1   0;]
b = [-c; c; f'; z81]
d = [zeros(1,24) 1];

In [27]:
model = Model()
set_optimizer(model, GLPK.Optimizer)
y = @variable(model, [1:14], base_name="y", Int);
z = @variable(model, [1:11], base_name="z");
@objective(model, Min, z[11]);

In [28]:
for i in 1:14
    @constraint(model, y[i]>=0)
end
for i in 1:11
    @constraint(model, z[i]>=0)
end
@constraint(model, A_h[:, 1:14]*y + A_h[:, 15:25]*z.==b);

LoadError: BoundsError: attempt to access 9×17 Array{Float64,2} at index [1:9, 15:25]

In [29]:
optimize!(model)

In [30]:
optimal_solution = value.(y)

14-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

With an extremely accurate m/z value, we can converge to a correct answer.
<br> In function, we have:

In [31]:
using LinearAlgebra
using JuMP
using GLPKMathProgInterface
function pred(c, f)
    a = [12 1.007825 15.994915 14.003074 30.973762 31.972071 34.968853]'
    I = diagm(0=>fill(1, 7))
    z17 = zeros(1,7)
    z71 = zeros(7,1)
    z81 = zeros(8,1)
    
    A   = [-a'                            z17  -1  0   0   0   0   0   0   0   0   0   1;
            a'                            z17  0   -1  0   0   0   0   0   0   0   0   1;
            I                             I    z71 z71 z71 z71 z71 z71 z71 z71 z71 z71 z71;
            -0.2  1   0   0   0   0   0   z17  0   0   -1  0   0   0   0   0   0   0   0;
            -3.1  1   0   0   0   0   0   z17  0   0   0   1   0   0   0   0   0   0   0;
            -1.2  0   1   0   0   0   0   z17  0   0   0   0   1   0   0   0   0   0   0;
            -1.3  0   0   1   0   0   0   z17  0   0   0   0   0   1   0   0   0   0   0;
            -0.3  0   0   0   1   0   0   z17  0   0   0   0   0   0   1   0   0   0   0;
            -0.8  0   0   0   0   1   0   z17  0   0   0   0   0   0   0   1   0   0   0;
            -0.8  0   0   0   0   0   1   z17  0   0   0   0   0   0   0   0   1   0   0;
            -2.27 0   1   1   1   1   1   z17  0   0   0   0   0   0   0   0   0   1   0;]
    b = [-c; c; f'; z81]
    model = Model()
    set_optimizer(model, GLPK.Optimizer)
    y = @variable(model, [1:14], base_name="y", Int);
    z = @variable(model, [1:11], base_name="z");
    @objective(model, Min, z[11]);
    for i in 1:14
        @constraint(model, y[i]>=0)
    end
    for i in 1:11
        @constraint(model, z[i]>=0)
    end
    @constraint(model, A_h[:, 1:14]*y + A_h[:, 15:25]*z.==b);
    optimize!(model)
    return optimal_solution = value.(y)[1:7]
end

pred (generic function with 1 method)

In [32]:
f = [16 22 3 1 0 0 0]
a = [233.1047 205.0859 175.0753 149.0233 135.0440 126.1277 121.0293 84.0810]
p = [13 15 3 1 0 0 0;
     12 13 3 0 0 0 0;
     11 11 2 0 0 0 0;
     8  5  3 0 0 0 0;
     8  7  2 0 0 0 0;
     8  16 0 1 0 0 0;
     7  5  2 0 0 0 0;
     5  10 0 1 0 0 0;]
for i in 1:8
    c = a[i]
    t = p[i,:]
    r = pred(a[i]+0.000549, f)
    r = convert(Array{Int64,1}, r)
    if r == t
        println("C",t[1],"H",t[2],"O",t[3],"N",t[4],"P",t[5],"S",t[6],"Cl",t[7]," is correctly predicted")
    end
    if r != t
        println("C",t[1],"H",t[2],"O",t[3],"N",t[4],"P",t[5],"S",t[6],"Cl",t[7]," is misprected as", 
                "C",r[1],"H",r[2],"O",r[3],"N",r[4],"P",r[5],"S",r[6],"Cl",r[7])
    end
end

LoadError: BoundsError: attempt to access 9×17 Array{Float64,2} at index [1:9, 15:25]

In [33]:
f = [15 23 5 6 0 1 0]
a = [298.0942 264.0908 250.0918 163.0411 145.0302 136.0614 121.0505 119.0375]
p = [11 16 3 5 0 1 0;
     11 14 1 5 0 1 0;
     10 12 3 5 0 0 0;
     6  11 3 0 0 1 0;
     6  9  2 0 0 1 0;
     5  6  0 5 0 0 0;
     5  5  0 4 0 0 0;
     7  5  1 1 0 0 0;]
for i in 1:8
    c = a[i]
    t = p[i,:]
    r = pred(a[i]+0.000549, f)
    r = convert(Array{Int64,1}, r)
    if r == t
        println("C",t[1],"H",t[2],"O",t[3],"N",t[4],"P",t[5],"S",t[6],"Cl",t[7]," is correctly predicted")
    end
    if r != t
        println("C",t[1],"H",t[2],"O",t[3],"N",t[4],"P",t[5],"S",t[6],"Cl",t[7]," is misprected as ", 
                "C",r[1],"H",r[2],"O",r[3],"N",r[4],"P",r[5],"S",r[6],"Cl",r[7])
    end
end

LoadError: BoundsError: attempt to access 9×17 Array{Float64,2} at index [1:9, 15:25]

In [34]:
f = [16 24 5 1 0 0 0]
a = [276.1600 251.1154 248.1650 207.1023 175.0759 151.0395 142.1229 137.0602 100.0759 88.0761 70.0654]
p = [16 22 3 1 0 0 0;
     13 17 4 1 0 0 0;
     15 22 2 1 0 0 0;
     12 15 3 0 0 0 0;
     11 11 2 0 0 0 0;
     8  7  3 0 0 0 0;
     8  16 1 1 0 0 0;
     8  9  2 0 0 0 0;
     5  10 1 1 0 0 0;
     4  10 1 1 0 0 0;
     4  8  0 1 0 0 0;]
for i in 1:11
    c = a[i]
    t = p[i,:]
    r = pred(a[i]+0.000549, f)
    r = convert(Array{Int64,1}, r)
    if r == t
        println("C",t[1],"H",t[2],"O",t[3],"N",t[4],"P",t[5],"S",t[6],"Cl",t[7]," is correctly predicted")
    end
    if r != t
        println("C",t[1],"H",t[2],"O",t[3],"N",t[4],"P",t[5],"S",t[6],"Cl",t[7]," is misprected as ", 
                "C",r[1],"H",r[2],"O",r[3],"N",r[4],"P",r[5],"S",r[6],"Cl",r[7])
    end
end

LoadError: BoundsError: attempt to access 9×17 Array{Float64,2} at index [1:9, 15:25]

In [35]:
f = [10 15 13 5 3 0 0]
a = [426.0217 408.0105 272.9530 238.8950 176.9390 158.9246 134.0464 96.9683 78.9587]
p = [10 14 10 5 2 0 0;
     10 12 9  5 2 0 0;
     5  7  9  0 2 0 0;
     0  2  9  0 3 0 0;
     0  3  7  0 2 0 0;
     0  1  6  0 2 0 0;
     5  4  0  5 0 0 0;
     0  2  4  0 1 0 0;
     0  0  3  0 1 0 0;]
for i in 1:9
    c = a[i]
    t = p[i,:]
    r = pred(a[i]-0.000549, f)
    r = convert(Array{Int64,1}, r)
    if r == t
        println("C",t[1],"H",t[2],"O",t[3],"N",t[4],"P",t[5],"S",t[6],"Cl",t[7]," is correctly predicted")
    end
    if r != t
        println("C",t[1],"H",t[2],"O",t[3],"N",t[4],"P",t[5],"S",t[6],"Cl",t[7]," is misprected as ", 
                "C",r[1],"H",r[2],"O",r[3],"N",r[4],"P",r[5],"S",r[6],"Cl",r[7])
    end
end

LoadError: BoundsError: attempt to access 9×17 Array{Float64,2} at index [1:9, 15:25]